In [1]:
import os
from torchvision import datasets, transforms, models
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.models import resnet18

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device in use:", device)

# Paths
id_path = r"D:\ODD PROJECT\DATA\BDD\scene_classification\ID"

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet standards
                         std=[0.229, 0.224, 0.225])
])

# Load dataset (we treat all as one class here)
dataset = datasets.ImageFolder(root=r"D:\ODD PROJECT\DATA\BDD\scene_classification", transform=transform)

# Create labels manually: 0 for ID, 1 for OOD (we’ll use OOD later)
# For now: just train on class 0 (id)
id_only = [img for img in dataset.samples if "id" in img[0].lower()]
dataset.samples = id_only

# Dataloader
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Load pretrained ResNet-18
# Load ResNet-18 without weights (base model)
model = resnet18()

# Load manually downloaded weights
model.load_state_dict(torch.load(r"D:\ODD PROJECT\weights\resnet18-f37072fd.pth"))

# Modify the final layer
import torch.nn as nn
model.fc = nn.Linear(model.fc.in_features, 1)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
epochs = 5
for epoch in range(epochs):
    total_loss = 0
    model.train()
    for imgs, _ in loader:
        imgs = imgs.to(device)
        labels = torch.zeros(imgs.size(0), 1).to(device)  # All labels are 0 (ID)

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

# Save model
os.makedirs("../models", exist_ok=True)
torch.save(model.state_dict(), "../models/resnet18_id_only.pth")
print(" Model trained and saved.")


Device in use: cpu
Epoch [1/5], Loss: 8.5462
Epoch [2/5], Loss: 3.7063
Epoch [3/5], Loss: 1.7012
Epoch [4/5], Loss: 0.9839
Epoch [5/5], Loss: 0.6665
 Model trained and saved.
